In [1]:
import tensorflow as tf
import os
from PIL import Image

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
tf.__version__

'2.5.0-rc1'

In [3]:
base = "../data/data_dir/"
images = os.listdir(os.path.join(base,"Images/"))
masks = os.listdir(os.path.join(base, "Masks/"))
images.sort()
masks.sort()

In [9]:
def image_example(image_string, label_string):
    image_shape = tf.image.decode_png(image_string).shape

    feature = {
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_string])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b"png"])),
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_shape[0]])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_shape[1]])),
        'image/segmentation/class/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[label_string])),
        'image/segmentation/class/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b"png"])),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [11]:
record_file = 'trainseg.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for image, mask in zip(images, masks):
        img = open(os.path.join(base,"Images/",image), 'rb').read()
        msk = open(os.path.join(base,"Masks/",mask), 'rb').read()
        example = image_example(img, msk)
        writer.write(example.SerializeToString())
!du -sh {record_file}

 48M	trainseg.tfrecords


In [299]:
def _parse_function(example_proto):
    """Function to parse the example proto.

    Args:
      example_proto: Proto in the format of tf.Example.

    Returns:
      A dictionary with parsed image, label, height, width and image name.

    Raises:
      ValueError: Label is of wrong shape.
    """

    # Currently only supports jpeg and png.
    # Need to use this logic because the shape is not known for
    # tf.image.decode_image and we rely on this info to
    # extend label if necessary.
    def _decode_image(content, channels):
      return tf.cond(
          tf.image.is_jpeg(content),
          lambda: tf.image.decode_jpeg(content, channels),
          lambda: tf.image.decode_png(content, channels))

    features = {
        'image/encoded':
            tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/filename':
            tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/format':
            tf.io.FixedLenFeature((), tf.string, default_value='png'),
        'image/height':
            tf.io.FixedLenFeature((), tf.int64, default_value=0),
        'image/width':
            tf.io.FixedLenFeature((), tf.int64, default_value=0),
        'image/segmentation/class/encoded':
            tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/segmentation/class/format':
            tf.io.FixedLenFeature((), tf.string, default_value='png'),
    }

    parsed_features = tf.io.parse_single_example(example_proto, features)

    image = _decode_image(parsed_features['image/encoded'], channels=3)
    print(image.shape)

    return image

IndentationError: expected an indented block (<ipython-input-299-719c169bd973>, line 2)

In [276]:
raw_image_dataset = tf.data.TFRecordDataset('../data/trainseg.tfrecords')

In [277]:
it = iter(raw_image_dataset.map(_parse_function))

(None, None, 3)


In [298]:
next(it)

<tf.Tensor: shape=(240, 320, 3), dtype=uint8, numpy=
array([[[147, 178, 180],
        [147, 178, 180],
        [147, 178, 180],
        ...,
        [ 60,  80,  87],
        [ 62,  82,  89],
        [ 63,  83,  90]],

       [[169, 199, 201],
        [169, 199, 201],
        [169, 199, 201],
        ...,
        [ 76,  96, 103],
        [ 78,  98, 105],
        [ 78,  98, 105]],

       [[186, 214, 217],
        [186, 214, 217],
        [186, 214, 217],
        ...,
        [ 89, 109, 116],
        [ 90, 110, 117],
        [ 90, 110, 117]],

       ...,

       [[ 58,  57,  62],
        [ 58,  57,  62],
        [ 57,  56,  61],
        ...,
        [ 36,  36,  36],
        [ 36,  36,  36],
        [ 37,  37,  37]],

       [[ 60,  59,  64],
        [ 59,  58,  63],
        [ 58,  57,  62],
        ...,
        [ 38,  38,  38],
        [ 38,  38,  38],
        [ 38,  38,  38]],

       [[ 60,  59,  64],
        [ 60,  59,  64],
        [ 59,  58,  63],
        ...,
        [ 39,  39,  3

In [ ]:
PYTHONPATH=.:./slim python3.7 deeplab/train.py \
    --logtostderr \
    --training_number_of_steps=30000 \
    --train_split="train" \
    --model_variant="xception_65" \
    --atrous_rates=6 \
    --atrous_rates=12 \
    --atrous_rates=18 \
    --output_stride=16 \
    --decoder_output_stride=4 \
    --train_crop_size=513 \
    --train_crop_size=513 \
    --train_batch_size=1 \
    --dataset="trainseg" \
    --tf_initial_checkpoint=${PATH_TO_INITIAL_CHECKPOINT} \
    --train_logdir=/home/ubuntuadm/trainseg/train_log \
    --dataset_dir=/home/ubuntuadm/trainseg/data/